In [4]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

data = pd.read_csv('as1-bank.csv')


X = data.drop('y', axis=1)
y = data['y']


categorical_cols = ['housing', 'loan', 'default']


numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ~X.columns.isin(categorical_cols)),
        ('cat', categorical_transformer, categorical_cols)
    ])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier(max_iter=1000, solver='sgd', random_state=1))
])


param_grid = {
    'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
    'classifier__activation': ['logistic', 'tanh', 'relu']
}


grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)


print("Best parameters found:")
print(grid_search.best_params_)
print("Best cross-validation score:")
print(grid_search.best_score_)


y_pred = grid_search.best_estimator_.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)


print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')


Best parameters found:
{'classifier__activation': 'tanh', 'classifier__hidden_layer_sizes': (100, 50)}
Best cross-validation score:
0.8428216321317601
Accuracy: 0.8419375398342893
Classification Report:
              precision    recall  f1-score   support

          no       0.88      0.93      0.90      1207
         yes       0.70      0.56      0.62       362

    accuracy                           0.84      1569
   macro avg       0.79      0.74      0.76      1569
weighted avg       0.83      0.84      0.84      1569

